In [1]:
import os
import numpy as np 
import struct
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
from mindvision.dataset import Mnist
from mindvision.classification.models import lenet
import mindspore.nn as nn

from mindspore.train.callback import ModelCheckpoint, CheckpointConfig

from mindvision.engine.callback import LossMonitor
from mindspore.train import Model


In [2]:
# 下载并处理MNIST数据集
download_train = Mnist(path="./data", split="train", batch_size=32, repeat_num=1, shuffle=True, resize=32, download=True)

download_eval = Mnist(path="./data", split="test", batch_size=32, resize=32, download=True)

dataset_train = download_train.run()
dataset_eval = download_eval.run()

9913344B [00:00, 13578481.64B/s]                              
29696B [00:00, 4242159.72B/s]           
1649664B [00:00, 3609769.80B/s]                             
5120B [00:00, 5094860.37B/s]           
[WARNING] ME(4016:2820,MainProcess):2023-06-05-11:10:57.570.545 [mindspore\dataset\core\validator_helpers.py:806] 'Resize' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'Resize' from mindspore.dataset.vision instead.
[WARNING] ME(4016:2820,MainProcess):2023-06-05-11:10:57.571.539 [mindspore\dataset\core\validator_helpers.py:806] 'Rescale' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'Rescale' from mindspore.dataset.vision instead.
[WARNING] ME(4016:2820,MainProcess):2023-06-05-11:10:57.573.540 [mindspore\dataset\core\validator_helpers.py:806] 'Rescale' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be rem

In [7]:
network = lenet(num_classes=10, pretrained=False)
# 定义损失函数
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')

# 定义优化器函数
net_opt = nn.Momentum(network.trainable_params(), learning_rate=0.01, momentum=0.9)

# 设置模型保存参数，模型训练保存参数的step为1875
config_ck = CheckpointConfig(save_checkpoint_steps=1875, keep_checkpoint_max=10)

# 应用模型保存参数
ckpoint = ModelCheckpoint(prefix="lenet", directory="./lenet", config=config_ck)

# 初始化模型参数
model = Model(network, loss_fn=net_loss, optimizer=net_opt, metrics={'accuracy'})

# 训练网络模型，并保存为lenet-1_1875.ckpt文件
model.train(10, dataset_train, callbacks=[ckpoint, LossMonitor(0.01, 1875)])


Epoch:[  0/ 10], step:[ 1875/ 1875], loss:[0.015/1.432], time:23.047 ms, lr:0.01000
Epoch time: 31107.867 ms, per step time: 16.591 ms, avg loss: 1.432
Epoch:[  1/ 10], step:[ 1875/ 1875], loss:[0.059/0.086], time:20.001 ms, lr:0.01000
Epoch time: 32149.162 ms, per step time: 17.146 ms, avg loss: 0.086
Epoch:[  2/ 10], step:[ 1875/ 1875], loss:[0.036/0.051], time:20.503 ms, lr:0.01000
Epoch time: 27639.373 ms, per step time: 14.741 ms, avg loss: 0.051
Epoch:[  3/ 10], step:[ 1875/ 1875], loss:[0.105/0.039], time:26.217 ms, lr:0.01000
Epoch time: 29597.785 ms, per step time: 15.785 ms, avg loss: 0.039
Epoch:[  4/ 10], step:[ 1875/ 1875], loss:[0.123/0.032], time:15.999 ms, lr:0.01000
Epoch time: 36617.920 ms, per step time: 19.530 ms, avg loss: 0.032
Epoch:[  5/ 10], step:[ 1875/ 1875], loss:[0.019/0.027], time:35.516 ms, lr:0.01000
Epoch time: 38052.165 ms, per step time: 20.294 ms, avg loss: 0.027
Epoch:[  6/ 10], step:[ 1875/ 1875], loss:[0.009/0.022], time:16.115 ms, lr:0.01000
Epoc

In [10]:
acc = model.eval(dataset_eval)

print("the test-accuracy is "+str(acc) )


the test-accuracy is {'accuracy': 0.9893830128205128}
